In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

import pymc3 as pm
import theano
import theano.tensor as tt
import arviz as az

In [2]:
def generate_data(alpha, beta, n=100, 
                  p_r={'high_var': [.95, .05], 'low_var': [.5,.5]},
                  rs = np.array(([5.0, -495.0],[-5.0, 495.0],[10.0, -100.0],[-10.0, 100.0])),
                  sQ = np.zeros((4, 2))
                 ):
    
    # Need to denote both machine type and action
    
    # Pre-specify machines for each trial in a randomly balanced manner
    if n%4 != 0:
        print("Number of trials is not divisable by 4.\nCreating trials for %s trials."%(str(n-(n%4))))
        n = n-(n%4)
    
    machs = np.array([0,1,2,3])
    machs = np.tile(machs, int(n/4))
    np.random.shuffle(machs)
    
    # Initialize empty array that will be populated in the loop based on Q values
    acts = np.zeros(n, dtype=np.int)
    
    # Generate by coin flip for machine with differing probabilities and outcomes
    rews = np.zeros(n, dtype=np.int)

    # Stores the expected value for each of 4 machines in each trial for each action
    Qs = np.zeros((n, 4, 2))

    # Initialize Q table
    # Denotes expected value of each action
    # Should look like [0, 0] for each machine
    # *** The expected value of not playing should not change from 0! ***
    # Could these initial expected values/beliefs also be estimated from data?
    # E.g. what if kids have more optimistic priors about each machine though they learn at the same rate
    Q = sQ.copy()
    
    for i in range(n):
        
        cur_machine = machs[i]
        
        # Apply the Softmax transformation
        exp_Q = np.exp(beta*Q[cur_machine])
        prob_a = exp_Q / np.sum(exp_Q)

        # Simulate choice
        a = np.random.choice([0, 1], p=prob_a)
        
        # Simulate reward if machine is played
        if a == 1:
    
            # Before sampling reward determine which variance condition machine is in
            if cur_machine>1:
                cur_p = 'low_var'
            else:
                cur_p = 'high_var'

            # Sample reward for current machine given its reward probs and outcome options
            r = np.random.choice(rs[cur_machine], p = p_r[cur_p]) 
            
            # Update Q table only if the machine is played
            # And only the value of playing NOT of not playing
            Q[cur_machine][a] = Q[cur_machine][a] + alpha * (r - Q[cur_machine][a])
        
        # If the machine is not played then Q remains unchanged and no reward is received
        else:
            r = 0.0

        # Store values
        acts[i] = a
        rews[i] = r
        #Qs[i] = Q.copy()
        Qs[i] = Q

    return machs, acts, rews, Qs

In [3]:
def llik_td(x, *args):
    # Extract the arguments as they are passed by scipy.optimize.minimize
    alpha, beta = x
    machines, actions, rewards = args

    # Initialize values
    Q = np.zeros((4, 2))
    log_prob_actions = np.zeros(len(actions))

    for t, (m, a, r) in enumerate(zip(machines, actions, rewards)):
        
        # Apply the softmax transformation
        Q_ = Q[m] * beta
        log_prob_action = Q_ - scipy.special.logsumexp(Q_)

        # Store the log probability of the observed action
        log_prob_actions[t] = log_prob_action[a]

        # Update the Q values for the next trial
        # Q[a] = Q[a] + alpha * (r - Q[a])
        Q[m][a] = Q[m][a] + alpha * (r - Q[m][a])

    # Return the negative log likelihood of all observed actions
    return -np.sum(log_prob_actions[1:])

In [4]:
true_alpha = .3
true_beta = .5
n = 120
machines, actions, rewards, all_Qs = generate_data(true_alpha, true_beta, n)

In [5]:
llik_td([true_alpha, true_beta], *(machines, actions, rewards))

22.588324993517084

In [6]:
x0 = [true_alpha, true_beta]
#x0 = [.1, 3]
result = scipy.optimize.minimize(llik_td, x0, args=(machines, actions, rewards), method='BFGS')
print(result)
print('')
print(f'MLE: alpha = {result.x[0]:.2f} (true value = {true_alpha})')
print(f'MLE: beta = {result.x[1]:.2f} (true value = {true_beta})')

      fun: 21.716473138069635
 hess_inv: array([[ 0.13369853, -1.62111246],
       [-1.62111246, 19.77512604]])
      jac: array([-1.66893005e-06, -4.76837158e-07])
  message: 'Optimization terminated successfully.'
     nfev: 112
      nit: 23
     njev: 28
   status: 0
  success: True
        x: array([0.08698377, 1.23184674])

MLE: alpha = 0.09 (true value = 0.3)
MLE: beta = 1.23 (true value = 0.5)


In [37]:
def llik_td_vectorized(x, *args):
    # Extract the arguments as they are passed by scipy.optimize.minimize
    alpha, beta = x
    machines, actions, rewards = args
    n = len(actions)

    # Create a list with the Q values of each trial
    #Qs = np.ones((n, 2), dtype=np.float)
    #Qs[0] = .5
    Qs = np.zeros((n, 4, 2), dtype=np.float)
    
    # The last Q values were never used, so there is no need to compute them
    for t, (m, a, r) in enumerate(zip(machines[:-1], actions[:-1], rewards[:-1])):
        Qs[t] = Qs[t-1]
        Qs[t, m, a] = Qs[t-1, m, a] + alpha * (r - Qs[t-1, m, a])
        Qs[t, m, 1-a] = Qs[t-1, m, 1-a]

    # Apply the softmax transformation in a vectorized way
    idx = list(zip(range(n-1),machines[:-1], actions[:-1]))
    obs_Qs = [Qs[i] for i in idx]
    Qs_ = np.array(obs_Qs) * beta
    log_prob_actions = Qs_ - scipy.special.logsumexp(Qs_, axis=1)[:, None]

    # Return the log_prob_actions for the observed actions
    log_prob_actions = log_prob_actions[np.arange(len(actions)), actions]
    return -np.sum(log_prob_actions[1:])

In [38]:
llik_td_vectorized([true_alpha, true_beta], *(machines, actions, rewards))

5557.576891122227

In [56]:
alpha = true_alpha
beta = true_beta
n = 10
Qs = np.zeros((n, 4, 2), dtype=np.float)
    
# The last Q values were never used, so there is no need to compute them
for t, (m, a, r) in enumerate(zip(machines[:n], actions[:n], rewards[:n])):  
    Qs[t] = Qs[t-1]
    Qs[t, m, a] = Qs[t-1, m, a] + alpha * (r - Qs[t-1, m, a])
    Qs[t, m, 1-a] = Qs[t-1, m, 1-a]

# Apply the softmax transformation in a vectorized way
obs_Qs = [Qs[i] for i in idx]
Qs_ = np.array(obs_Qs) * beta
#log_prob_actions = Qs_ - scipy.special.logsumexp(Qs_, axis=1)[:, None]

# Return the log_prob_actions for the observed actions
#log_prob_actions = log_prob_actions[np.arange(len(actions)), machines, actions]